Carga de archivos del SAC

Diariamente robot descarga achivos y los coloco en <span style="color: rgb(68, 140, 39); font-family: Consolas, &quot;Courier New&quot;, monospace; font-size: 12px; white-space: pre;">C:\Cargas</span>

In [ ]:
use SAC_MAR
 
IF OBJECT_ID('TEMPDB..#TEMP_FILES') IS NOT NULL DROP TABLE #TEMP_FILES

CREATE TABLE #TEMP_FILES
(
FileName VARCHAR(MAX),
DEPTH VARCHAR(MAX),
[FILE] VARCHAR(MAX)
)
 
INSERT INTO #TEMP_FILES
EXEC master.dbo.xp_DirTree 'C:\Cargas\',1,1


DECLARE @FILENAME VARCHAR(MAX),@SQL VARCHAR(MAX)
 
select * from #TEMP_FILES

--IF OBJECT_ID('TEMPDB..#TEMP_RESULTS') IS NOT NULL DROP TABLE #TEMP_RESULTS

--CREATE TABLE #TEMP_RESULTS
--(
--[FECHA] [varchar](50) NULL,
--	[TD_PROSA_ACQ_ENTITY] [varchar](50) NULL,
--	[TTR_NUMERO] [varchar](50) NULL,
--	[CANTIDAD] [varchar](50) NULL,
--	[IMPORTE] [varchar](50) NULL,
--	[CUOTA COMERCIO] [varchar](50) NULL,
--	[IVA CUOTA COMERCIO] [varchar](50) NULL,
--	[CUOTA INTERCAMBIO] [varchar](50) NULL,
--	[IVA CUOTA INTERCAMBIO] [varchar](50) NULL,
--	[PREFIJO] [varchar](50) NULL,
--	[NO_COMERCIO] [varchar](50) NULL,
--	[COMERCIO] [varchar](50) NULL
--)
 
WHILE EXISTS(SELECT * FROM #TEMP_FILES)
BEGIN
   BEGIN TRY
      SET @FILENAME = (SELECT TOP 1 FileName FROM #TEMP_FILES)
      SET @SQL = 'BULK INSERT  SAC_MAR.[dbo].[SAC_carga]
      FROM ''C:\Cargas\' + @FILENAME +'''
      WITH (FIRSTROW = 2, FIELDTERMINATOR = ''|'', ROWTERMINATOR = ''0x0a'');'

      PRINT @SQL
      EXEC(@SQL)
   END TRY
   BEGIN CATCH
      PRINT 'Failed processing : ' + @FILENAME
   END CATCH

   DELETE FROM #TEMP_FILES WHERE FileName = @FILENAME
END 

--SELECT * FROM #TEMP_RESULTS

--drop table #TEMP_RESULTS

--truncate table SAC_ene.dbo.SAC_carga


Una vez cargada la info actualizo la tabla de carga

In [ ]:
use sac_mar

DECLARE @fecha varchar(20) = '20230321'
DECLARE @fecha_archivo1 varchar(20)='21/03/2023 0:00'
DECLARE @fecha_archivo2 varchar(20)='21/03/2023 00:00'


update sac_mar.dbo.[SAC_carga]
set fecha=@fecha
where fecha=@fecha_archivo1


update sac_mar.dbo.[SAC_carga]
set fecha=@fecha
where fecha=@fecha_archivo2

update [SAC_carga]
set PREFIJO=REPLACE(prefijo,',','')

update [SAC_carga]
set PREFIJO=REPLACE(prefijo,'.','')

update [SAC_carga]
set PREFIJO=REPLACE(prefijo,'E7','')

update [SAC_carga]
set PREFIJO=prefijo+'0'
where len(prefijo)=5

update [SAC_carga]
set PREFIJO=prefijo+'00'
where len(prefijo)=4

update [SAC_carga]
set PREFIJO=prefijo+'000'
where len(prefijo)=3

Inserto registros de la tabla

In [ ]:

INSERT INTO sac_mar.dbo.[INFO_SAC]
           ([FECHA]
           ,[id_adquirente]
           ,[cve_txn]
           ,[cantidad]
           ,[IMPORTE]
           ,[CUOTA_COMERCIO]
           ,[IVA_CUOTA_COMERCIO]
           ,[CUOTA_INTERCAMBIO]
           ,[IVA_CUOTA_INTERCAMBIO]
           ,[PREFIJO]
           ,[NO_COMERCIO]
           ,[COMERCIO])
			SELECT fecha
				  ,[TD_PROSA_ACQ_ENTITY]
				  ,[TTR_NUMERO]
				  ,replace([CANTIDAD],',','')
				  ,replace([IMPORTE],',','.')
				  ,replace([CUOTA COMERCIO],',','.')
				  ,replace([IVA CUOTA COMERCIO],',','.')
				  ,replace([CUOTA INTERCAMBIO],',','.')
				  ,replace([IVA CUOTA INTERCAMBIO],',','.')
				  ,replace([PREFIJO],',0','')
				  ,[NO_COMERCIO]
				  ,[COMERCIO]
			  FROM sac_mar.dbo.[SAC_carga]


Borro tabla de carga

In [ ]:
truncate table SAC_mar.dbo.SAC_carga

Validar cifras control

In [ ]:
use SAC_mar

select fecha, count(1) as registros, sum(cantidad) as txns, sum(importe) as total, sum(importe)/sum(cantidad) as pagare_promedio
from INFO_SAC
 where id_adquirente=747
group by fecha
order by fecha


## CARGA DE iNTERREDES

In [ ]:
use SAC_MAR
 
IF OBJECT_ID('TEMPDB..#TEMP_FILES') IS NOT NULL DROP TABLE #TEMP_FILES

CREATE TABLE #TEMP_FILES
(
FileName VARCHAR(MAX),
DEPTH VARCHAR(MAX),
[FILE] VARCHAR(MAX)
)
 
INSERT INTO #TEMP_FILES
EXEC master.dbo.xp_DirTree 'C:\Cargas\',1,1
 



DECLARE @FILENAME VARCHAR(MAX),@SQL VARCHAR(MAX)
 
select * from #TEMP_FILES

--IF OBJECT_ID('TEMPDB..#TEMP_RESULTS') IS NOT NULL DROP TABLE #TEMP_RESULTS

--CREATE TABLE #TEMP_RESULTS
--(
--[FECHA] [varchar](50) NULL,
--	[TD_PROSA_ACQ_ENTITY] [varchar](50) NULL,
--	[TTR_NUMERO] [varchar](50) NULL,
--	[CANTIDAD] [varchar](50) NULL,
--	[IMPORTE] [varchar](50) NULL,
--	[CUOTA COMERCIO] [varchar](50) NULL,
--	[IVA CUOTA COMERCIO] [varchar](50) NULL,
--	[CUOTA INTERCAMBIO] [varchar](50) NULL,
--	[IVA CUOTA INTERCAMBIO] [varchar](50) NULL,
--	[PREFIJO] [varchar](50) NULL,
--	[NO_COMERCIO] [varchar](50) NULL,
--	[COMERCIO] [varchar](50) NULL
--)
 
WHILE EXISTS(SELECT * FROM #TEMP_FILES)
BEGIN
   BEGIN TRY
      SET @FILENAME = (SELECT TOP 1 FileName FROM #TEMP_FILES)
      SET @SQL = 'BULK INSERT  SAC_MAR.[dbo].[SAC_carga]
      FROM ''C:\Cargas\' + @FILENAME +'''
      WITH (FIRSTROW = 2, FIELDTERMINATOR = ''|'', ROWTERMINATOR = ''0x0a'');'

      PRINT @SQL
      EXEC(@SQL)
   END TRY
   BEGIN CATCH
      PRINT 'Failed processing : ' + @FILENAME
   END CATCH

   DELETE FROM #TEMP_FILES WHERE FileName = @FILENAME
END 

--SELECT * FROM #TEMP_RESULTS

--drop table #TEMP_RESULTS

--truncate table SAC_ene.dbo.SAC_carga


## ACTUALIZAR

In [ ]:
use sac_mar

DECLARE @fecha varchar(20) = '20230321'
DECLARE @fecha_archivo1 varchar(20)='21/03/2023 0:00'
DECLARE @fecha_archivo2 varchar(20)='21/03/2023 00:00'


update sac_mar.dbo.[SAC_carga]
set fecha=@fecha
where fecha=@fecha_archivo1


update sac_mar.dbo.[SAC_carga]
set fecha=@fecha
where fecha=@fecha_archivo2

update [SAC_carga]
set PREFIJO=REPLACE(prefijo,',','')

update [SAC_carga]
set PREFIJO=REPLACE(prefijo,'.','')

update [SAC_carga]
set PREFIJO=REPLACE(prefijo,'E7','')

update [SAC_carga]
set PREFIJO=prefijo+'0'
where len(prefijo)=5

update [SAC_carga]
set PREFIJO=prefijo+'00'
where len(prefijo)=4

update [SAC_carga]
set PREFIJO=prefijo+'000'
where len(prefijo)=3

## INSERTAR

In [ ]:

INSERT INTO sac_mar.dbo.[INFO_SAC_INTERREDES]
           ([FECHA]
           ,[id_adquirente]
           ,[cve_txn]
           ,[cantidad]
           ,[IMPORTE]
           ,[CUOTA_COMERCIO]
           ,[IVA_CUOTA_COMERCIO]
           ,[CUOTA_INTERCAMBIO]
           ,[IVA_CUOTA_INTERCAMBIO]
           ,[PREFIJO]
           ,[NO_COMERCIO]
           ,[COMERCIO])
			SELECT fecha
				  ,[TD_PROSA_ACQ_ENTITY]
				  ,[TTR_NUMERO]
				  ,replace([CANTIDAD],',','')
				  ,replace([IMPORTE],',','.')
				  ,replace([CUOTA COMERCIO],',','.')
				  ,replace([IVA CUOTA COMERCIO],',','.')
				  ,replace([CUOTA INTERCAMBIO],',','.')
				  ,replace([IVA CUOTA INTERCAMBIO],',','.')
				  ,replace([PREFIJO],',0','')
				  ,[NO_COMERCIO]
				  ,[COMERCIO]
			  FROM sac_mar.dbo.[SAC_carga]


## BORRO TABLA DE CARGA

In [ ]:
truncate table SAC_mar.dbo.SAC_carga

## VALIDO CIFRAS CONTROL

In [ ]:
use SAC_mar

select fecha, count(1) as registros, sum(cantidad) as txns, sum(importe) as total, sum(importe)/sum(cantidad) as pagare_promedio
from INFO_SAC_INTERREDES
group by fecha
order by fecha